In [1]:
# we will use sqlite3 python module in this tutorial
# Python comes with sqlite3

In [2]:
#### Creating basic SQlite Tables ####

In [3]:
# Types in sqlite
# NULL
# INTEGER
# REAL
# TEXT
# BLOB

In [12]:
import sqlite3

conn = sqlite3.connect("my_friends.db") # will cretae one if not exists
#create curson object
c = conn.cursor()
#execute some sql
#c.execute("CREATE TABLE friends(first_name TEXT, last_name TEXT, closeness INTEGER)")
#insert_query = '''INSERT INTO friends VALUES ("Kanu", "Hasini", 7)'''
#form_first = "Dhinesh"
#insert_query = f"INSERT INTO friends (first_name) VALUES ('{form_first}')"
#insert_query = f"INSERT INTO friends (first_name) VALUES (?)"
data = ("Dhineshbabu", "Muthuraj", 30)
insert_query = f"INSERT INTO friends VALUES (?,?,?)"
c.execute(insert_query, data)
#commit changes
conn.commit()
conn.close()

In [ ]:
#### BULK inserts with python ####

In [14]:
import sqlite3

conn = sqlite3.connect("my_friends.db") # will cretae one if not exists
#create curson object
c = conn.cursor()

people = [
	("Roald","Amundsen", 5),
	("Rosa", "Parks", 8),
	("Henry", "Hudson", 7),
	("Neil","Armstrong", 7),
	("Daniel", "Boone", 3)]

c.executemany("INSERT INTO friends VALUES (?,?,?)", people)
#commit changes
conn.commit()
conn.close()

In [15]:
### selecting with python ###

In [17]:
import sqlite3

conn = sqlite3.connect("my_friends.db") # will cretae one if not exists
#create curson object
c = conn.cursor()
c.execute("SELECT * FROM friends")
for result in c:
    print(result)
#commit changes
conn.commit()
conn.close()

('Kanu', 'Hasini', 7)
('Bru', None, None)
('Dhinesh', None, None)
('Dhineshbabu', 'Muthuraj', 30)
('Roald', 'Amundsen', 5)
('Rosa', 'Parks', 8)
('Henry', 'Hudson', 7)
('Neil', 'Armstrong', 7)
('Daniel', 'Boone', 3)


In [21]:
### another way of selecting using ftechall() method ###

import sqlite3

conn = sqlite3.connect("my_friends.db") # will cretae one if not exists
#create curson object
c = conn.cursor()
c.execute("SELECT * FROM friends WHERE first_name IS 'Rosa'")
print(c.fetchall())
#commit changes
conn.commit()
conn.close()

[('Rosa', 'Parks', 8)]


In [23]:
###fetchone() method ###
import sqlite3

conn = sqlite3.connect("my_friends.db") # will cretae one if not exists
#create curson object
c = conn.cursor()
c.execute("SELECT * FROM friends WHERE first_name IS 'Rosa'")
print(c.fetchone())
#commit changes
conn.commit()
conn.close()

('Rosa', 'Parks', 8)


In [24]:
############### SQL INJECTION ##########

In [30]:
import sqlite3

conn = sqlite3.connect("users.db")
c = conn.cursor()

#query = "CREATE TABLE users (username TEXT, password TEXT)"
users = [
	("Dhinesh","indian123"),
	("Kanu", "sample132"),
	("Bru", "hello123")]

c.executemany("INSERT INTO users VALUES (?,?)", users)
conn.commit()
conn.close()

In [33]:
import sqlite3

conn = sqlite3.connect("users.db")
c = conn.cursor()

#query = "CREATE TABLE users (username TEXT, password TEXT)"
u = input("Please enter your username...")
p = input("Please enter your password...")
query = f"SELECT * FROM users WHERE username='{u}' AND password='{p}'"
# if i give password like ' OR 1=1-- this will return the successful data.
# replace with ? for the the paramters and send the data in the c.execute
c.execute(query)
result = c.fetchone()
if(result):
    print("Welcome Back...")
else:
    print("Login Failed. Please check username and password...")
conn.commit()

Please enter your username... Dhinesh
Please enter your password... ' OR 1=1--


Welcome Back...


In [1]:
########## Scraping data to the database ######
# demo using http://books.toscrape.com/

In [17]:
import sqlite3
import requests
from bs4 import BeautifulSoup
    
def scrape_books(url):
    #Request URL
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    books = soup.find_all("article")
    all_books = []
    for book in books:
        book_data = (get_title(book), get_price(book), get_rating(book))
        all_books.append(book_data)
    save_books(all_books)
    
def save_books(all_books):
    connection  = sqlite3.connect("books.db")
    c = connection.cursor()
    c.execute('''CREATE TABLE books 
            (title TEXT, pricing REAL, rating INTEGER)''')
    c.executemany("INSERT INTO books VALUES (?,?,?)", all_books)
    connection.commit()
    connection.close()
    
def get_title(book):
    return book.find("h3").find("a")["title"]
        
def get_price(book)  :
    price = book.select(".price_color")[0].get_text()
    return float(price.replace("Â","").replace("£",""))

def get_rating(book):
    ratings = {"Zero":0, "One": 1, "Two":2, "Three":3, "Four": 4, "Five": 5}
    paragraph = book.select(".star-rating")[0]
    word = paragraph.get_attribute_list("class")[-1]
    return ratings[word]
    
scrape_books("http://books.toscrape.com/catalogue/category/books/history_32/index.html")